In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

C:\Users\Kyle\anaconda2\envs\py3-tf2.0\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading the dataset 

In [2]:
mnist_dataset,mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [3]:
#Seperating the data
mnist_train = mnist_dataset['train']
mnist_test = mnist_dataset['test']


#Taking 10% as validation data
num_validation = 0.1 * mnist_info.splits['train'].num_examples
num_validation = tf.cast(num_validation, tf.int64)

#Counting the number of  testing dataset
num_test = mnist_info.splits['test'].num_examples
num_test = tf.cast(num_test,tf.int64)

#scaleing the images
def scale(image,label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

##  Reshuffling and batching

In [4]:
BUFFER_SIZE = 10000

reshuffled_scaled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

##  Extracting the validation data

In [5]:
validation_data = reshuffled_scaled_train_and_validation_data.take(num_validation)

## Extracting the training data

In [6]:
training_data = reshuffled_scaled_train_and_validation_data.skip(num_validation)

In [7]:
BATCH_SIZE =100

training_data = training_data.batch(BATCH_SIZE)

In [8]:
validation_data = validation_data.batch(num_validation)
test_data = test_data.batch(num_test)

validation_input, validation_targets = next(iter(validation_data))

# The Model

In [9]:
inputs_size = 784
outputs_size =10
hidden_layers_size =150

model = tf.keras.Sequential([tf.keras.layers.Flatten(input_shape = (28,28,1)),
                             tf.keras.layers.Dense(hidden_layers_size,activation="relu"),
                             tf.keras.layers.Dense(hidden_layers_size,activation="relu"),
                             tf.keras.layers.Dense(outputs_size, activation = 'softmax')
])

##  Optimization

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

##  Training

In [11]:
NUM_EPOCH = 5
model.fit(training_data, epochs=NUM_EPOCH, validation_data=(validation_input,validation_targets), verbose = 2)

Epoch 1/5
540/540 - 4s - loss: 0.2998 - accuracy: 0.9139 - val_loss: 0.1546 - val_accuracy: 0.9573
Epoch 2/5
540/540 - 4s - loss: 0.1184 - accuracy: 0.9644 - val_loss: 0.0934 - val_accuracy: 0.9733
Epoch 3/5
540/540 - 4s - loss: 0.0803 - accuracy: 0.9752 - val_loss: 0.0710 - val_accuracy: 0.9788
Epoch 4/5
540/540 - 4s - loss: 0.0605 - accuracy: 0.9816 - val_loss: 0.0652 - val_accuracy: 0.9795
Epoch 5/5
540/540 - 3s - loss: 0.0495 - accuracy: 0.9846 - val_loss: 0.0511 - val_accuracy: 0.9842


##  Testing the model

In [12]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 15ms/step - loss: 0.0724 - accuracy: 0.9776
